In [1]:
import os
import sys
path_to_this_notebook = os.path.abspath('.')
path_to_project = path_to_this_notebook[:path_to_this_notebook.find('note')]
sys.path.append(path_to_project)

from src.save_and_load_tools.save_load_utils import save_grid, save_scenarios, load_grid
from src.surrogate_grids._grid_parallel_nodes import create_grid_parallel_nodes
from src.surrogate_grids._grid_single_node import create_grid_single_node
from src.generate_grid_examples.ieee_16 import generate_IEEE16
from src.scenario.scenario_generator import ScenarioGenerator
from src.plotting_tools.plot_grid import plot_grid

import pandas as pd
import pickle
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
meshed = False
grid_connected = False

v_min, v_max = 300, 400
load_p_min, load_p_max = 0, 10000
line_i_max = 30
line_g = 15
gen_p_max = 0
gen_p_min = -0

grid = generate_IEEE16(meshed, grid_connected,  v_min, v_max,
                       gen_p_min, gen_p_max, load_p_max, load_p_min, line_i_max, line_g)
# Scenarios params
n_scenarios = 6
t_start_hr = 0
t_end_hr = 24
ptu_size_minutes = 30
ptu_size_hr = ptu_size_minutes / 60
timesteps_hr = np.arange(0, t_end_hr + ptu_size_hr, ptu_size_hr)

charging_time_mean_bounds = 3.75, 3.76
charging_time_std_bounds = 1, 1.01
per_hour_demand_mean_bounds = 8500, 8501
per_hour_demand_std_bounds = 900, 901
std_price_factor=1e6


path_sessions = path_to_project + 'data/sessions_example.csv'
path_prices = path_to_project + 'data/power_price_example.csv'

scenario_generator = ScenarioGenerator(t_start_hr, t_end_hr, ptu_size_minutes, 
                                       charging_time_mean_bounds, charging_time_std_bounds, 
                                       per_hour_demand_mean_bounds, per_hour_demand_std_bounds,
                                       std_price_factor,  path_sessions, path_prices)

In [3]:
overwrite_grid = False
overwrite_scenarios = False
overwrite_plots = False

experiment_folder = 'IEEE16'
path_to_experiments = path_to_project + '/experiments/%s/' % experiment_folder
if not os.path.isdir(path_to_experiments):
    os.makedirs(path_to_experiments)
print(path_to_experiments)
print(os.listdir(path_to_experiments))

/home/gr1/Projects/EVCP_partially_observable_locations//experiments/IEEE16/
['IEEE16meshed_con']


In [4]:
# Specify parameters for the grids you want to create 

meshed = True
grid_connected = True


meshed_str = 'meshed' if meshed else 'radial'
external_grid_str = 'con' if grid_connected else 'disc'
experiment_name = 'IEEE16%s_%s' % (meshed_str, external_grid_str)
print('Name: ', experiment_name)
grid = generate_IEEE16(meshed, grid_connected, v_min, v_max, gen_p_min, gen_p_max,
                       load_p_max, load_p_min, line_i_max, line_g)

n_gens = grid.n_gens
n_loads = grid.n_loads
line_i_max_array = np.linspace(10, 40, 5)
gen_p_factors_array = np.linspace(1, 1, 1)
gen_p_min_array = np.array([-int(f * n_loads * load_p_max / n_gens) for f in gen_p_factors_array])
print('Ps: ', gen_p_min_array.round())
print('Is: ', line_i_max_array.round())
print('N grids: ', len(line_i_max_array) * len(gen_p_factors_array))

Name:  IEEE16meshed_con
Ps:  [-14444]
Is:  [10. 18. 25. 32. 40.]
N grids:  5


In [5]:
# Create and save the grids

for gen_p_min in gen_p_min_array:
    for line_i_max in line_i_max_array:
        grid = generate_IEEE16(meshed, grid_connected, v_min, v_max, gen_p_min, gen_p_max,
                               load_p_max, load_p_min, line_i_max, line_g)
        grid_parallel, _ = create_grid_parallel_nodes(grid)
        grid_single_cable, _ = create_grid_single_node(grid)
        grid_name = grid.name
        print('Grid: ', grid_name)
        path_to_grid_folder = path_to_experiments + '/%s/%s/' % (experiment_name, grid_name)
            
        if not os.path.isdir(path_to_grid_folder):
            print('Created folder')
            os.makedirs(path_to_grid_folder)
    
        grid_already_exists = 'grid.pickle' in os.listdir(path_to_grid_folder)
        
        if (not grid_already_exists )or overwrite_plots:
            plot_grid(grid, grid_name, bbox=(500, 500), margin=50, 
                      title_size=12, save=True, path_to_figures=path_to_grid_folder)
            plot_grid(grid_parallel, grid_name + '_parallel', bbox=(500, 500), margin=50, 
                      title_size=12, save=True, path_to_figures=path_to_grid_folder)
            plot_grid(grid_single_cable, grid_name + '_single-cable', bbox=(500, 500), margin=50, 
                      title_size=12, save=True, path_to_figures=path_to_grid_folder)
            print('Overwrote plots!')
            
        if not grid_already_exists:
            save_grid(grid, experiment_name, grid_name, path_to_experiments)
            print('Saved grid!')
            
        elif (grid_already_exists and overwrite_grid):
            save_grid(grid, experiment_name, grid_name, path_to_experiments)
            print('Overwrote existing grid!')
        else:
            grid = load_grid(experiment_name, grid_name, path_to_experiments)
            print('Grid found, skipping!')

        scenarios_already_exist = ('scenarios_t=%d.pickle' % (ptu_size_minutes) in os.listdir(path_to_grid_folder))
        scenarios = scenario_generator.generate(grid.load_inds, n_scenarios, 0, [])
        if not scenarios_already_exist:
            save_scenarios(scenarios, scenario_generator, experiment_name, grid_name, path_to_experiments)
            print('Saved scenarios!')
        elif (scenarios_already_exist and overwrite_scenarios):
            save_scenarios(scenarios, scenario_generator, experiment_name, grid_name, path_to_experiments)
            print('Overwrote existing scenarios!')
        else:
            print('Scenarios found, skipping!')

        print()

Grid:  IEEE16meshed_con_g=15_i-line-max=10_p-gen-min=-14444
Overwrote plots!
Saved grid!
Saved scenarios!

Grid:  IEEE16meshed_con_g=15_i-line-max=17_p-gen-min=-14444
Created folder
Overwrote plots!
Saved grid!
Saved scenarios!

Grid:  IEEE16meshed_con_g=15_i-line-max=25_p-gen-min=-14444
Created folder
Overwrote plots!
Saved grid!
Saved scenarios!

Grid:  IEEE16meshed_con_g=15_i-line-max=32_p-gen-min=-14444
Created folder
Overwrote plots!
Saved grid!
Saved scenarios!

Grid:  IEEE16meshed_con_g=15_i-line-max=40_p-gen-min=-14444
Created folder
Overwrote plots!
Saved grid!
Saved scenarios!

